<a href="https://colab.research.google.com/github/dcastf01/adversarial_project/blob/main/IRT/Extract_IRT_params_from_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.7/index.html
!pip install pytorch-pfn-extras timm
!pip install pytorch-ignite
!pip3 install --upgrade Pillow


Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.7/index.html
     |████████████████████████████████| 645kB 17.7MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Requirement already up-to-date: Pillow in /usr/local/lib/python3.7/dist-packages (8.1.1)


In [ ]:

%reload_ext autoreload
%autoreload

In [1]:
import os, sys, math
import numpy as np
from matplotlib import pyplot as plt
if 'google.colab' in sys.modules: # Colab-only Tensorflow version selector
  %tensorflow_version 2.x
  
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE
# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy for hardware
if tpu:
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print('Running on TPU ', tpu.master())  
elif len(gpus) > 0:
  #strategy = tf.distribute.MirroredStrategy(gpus) # this works for 1 to multiple GPUs
  print('Running on ', len(gpus), ' GPU(s) ')
  strategy=tf.distribute.OneDeviceStrategy(gpus[0])
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')

# How many accelerators do we have ?
print("Number of accelerators: ", strategy.num_replicas_in_sync)

Tensorflow version 2.4.1
Running on CPU
Number of accelerators:  1


In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [1]:
!git clone https://github.com/dcastf01/colab_utils.git
!git clone https://github.com/dcastf01/adversarial_project.git

Cloning into 'colab_utils'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 152 (delta 85), reused 30 (delta 14), pack-reused 0
Receiving objects: 100% (152/152), 163.61 KiB | 6.82 MiB/s, done.
Resolving deltas: 100% (85/85), done.
Cloning into 'adversarial_project'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 137 (delta 47), reused 98 (delta 22), pack-reused 0
Receiving objects: 100% (137/137), 1.80 MiB | 13.39 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [2]:
from colab_utils.utils.download_files import download_file_from_google_drive

In [ ]:
from colab_utils.utils.download_files import download_file_from_google_drive
from adversarial_project.data.IRT_features.dict_with_id_drive import id_drive_to_download
for name_file,id in id_drive_to_download.ids_dataset.items():
    if name_file[-4:]==".zip":
      name_folder=name_file[:-4]
      download_file_from_google_drive(id,destination=name_file)
      !unzip {name_file} -d {name_folder}
    else:

      download_file_from_google_drive(id,destination=name_file)


Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: imagenet_validation/ILSVRC2012_val_00040135.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040145.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040155.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040165.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040175.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040185.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040195.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040205.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040215.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040225.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040235.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040245.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040255.JPEG  
  inflating: imagenet_validation/ILSVRC2012_val_00040265.JPEG  
  inflating: imagenet_validation/ILSVRC2012

In [ ]:

import os
import timm
import shutil
from torchvision import datasets
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np

import torch
from torchvision import datasets
from torch.utils.data import (
    Dataset,
    DataLoader,
) 

In [ ]:
#@title Texto de título predeterminado
csv_with_info= "/content/csv_IRT.csv" #@param {type:"string"}

root_dir= "/content/imagenet_validation" #@param {type:"string"}

In [ ]:
from torchvision.datasets.folder import default_loader
from torch.nn import functional as F
class IRTFeaturesDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.csv_info = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.csv_info["norm_Dffclt"]=self.normalize(self.csv_info["Dffclt"].to_numpy())
        self.csv_info["norm_Dscrmn"]=self.normalize(self.csv_info["Dscrmn"].to_numpy())
        # self.max_dffclt=self.csv_info["Dffclt"].max()
        # self.min_dffclt=self.csv_info["Dffclt"].min()
        # self.norm_dffclt=np.linalg.norm(self.csv_info["Dffclt"].to_numpy())

        # print(self.csv_info.describe())
    def __len__(self):
        return len(self.csv_info)

    def normalize(self,x):
      x = np.asarray(x)
      # print(np.ptp(x))
      # return (x - x.min()) / (np.ptp(x))
      return x
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.root_dir,
                                self.csv_info.iloc[idx, 0]+".JPEG")
        image = default_loader(img_path)
        # landmarks = self.landmarks_frame.iloc[idx, 1:]
        # landmarks = np.array([landmarks])
        # landmarks = landmarks.astype('float').reshape(-1, 2)
        
        if self.transform:
            image = self.transform(image)
        # norm = np.linalg.norm(an_array)
        # normal_array = an_array/norm
        dffclt=self.csv_info['norm_Dffclt'][idx]
        # print(dffclt)
        dscrmn=self.csv_info['norm_Dscrmn'][idx]
        # dscrmn=F.normalize(dscrmn)
        # dscrmn= F.normalize(torch.tensor(self.csv_info.iloc[idx, 2]))

        sample = {'image': image,
                  "Dffclt": dffclt ,
                  "Dscrmn":  dscrmn,
                  'img_path': img_path}

        return sample


import torchvision.transforms as transforms

from timm.data.transforms_factory import transforms_imagenet_train,transforms_imagenet_eval
transformation=transforms_imagenet_train()
# transformations = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Resize(255),
#     transforms.CenterCrop(224),
    
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    
#     # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])


In [ ]:
#@title create dataset and defining some variables
percentage_train = 0.5 #@param {type:"slider", min:0, max:1, step:0.1}
percentage_test=0.3 #@param {type:"slider", min:0, max:1, step:0.1}
percentage_valid=0.2 #@param {type:"slider", min:0, max:1, step:0.1}
batch_size = 32 #@param {type:"integer"}
seed=1#@param {type:"integer"}

# dataset=datasets.ImageFolder(root=root_dir,transform=transformations) #add transforms
dataset=IRTFeaturesDataset(root_dir=root_dir,csv_file=csv_with_info,
                           transform=transformations,)

num_imgs=len(dataset)
split_train=num_imgs*percentage_train
split_test=num_imgs*percentage_test
split_valid=num_imgs*percentage_valid


train_set, test_set,valid_set = torch.utils.data.random_split(dataset, [int(split_train), int(split_test),int(split_valid+1)],generator=torch.Generator().manual_seed(seed))
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [ ]:
for x in test_loader:
  print(x)
  break

{'image': tensor([[[[-0.7826, -0.6974, -0.2946,  ..., -0.1841, -0.4040, -0.6381],
          [-0.7767, -0.6673, -0.2691,  ..., -0.3391, -0.5386, -0.6911],
          [-0.7711, -0.6392, -0.2353,  ..., -0.4563, -0.5789, -0.6997],
          ...,
          [ 0.5861,  0.5991,  0.5763,  ...,  0.3337,  0.2778,  0.2282],
          [ 0.6705,  0.6887,  0.5714,  ...,  0.3198,  0.2979,  0.2370],
          [ 0.6065,  0.6276,  0.5513,  ...,  0.3311,  0.3319,  0.2420]],

         [[-0.8912, -0.8290, -0.5541,  ..., -0.5123, -0.5965, -0.7942],
          [-0.9130, -0.8263, -0.5420,  ..., -0.6347, -0.7018, -0.8421],
          [-0.9113, -0.8118, -0.5297,  ..., -0.7504, -0.7433, -0.8462],
          ...,
          [ 0.5102,  0.5291,  0.5063,  ...,  0.0582, -0.0398, -0.0713],
          [ 0.5432,  0.5537,  0.4357,  ...,  0.0326, -0.0555, -0.0951],
          [ 0.4912,  0.4995,  0.4126,  ...,  0.0211, -0.0545, -0.1260]],

         [[-0.8740, -0.8577, -0.5772,  ..., -0.4356, -0.5466, -0.7226],
          [-0.8383, 

In [ ]:
from adversarial_project.pytorch.yamls import save_yaml
from adversarial_project.pytorch.clasification_flag_class import Flags

flags_dict = {
    "debug": False,  # Change to True for fast debug run!
    "outdir": "results/tmp_debug",
    # Data
    "imgdir_name": "vinbigdata-chest-xray-resized-png-256x256",
    # Model
    "model_name": "resnet50",
    "model_mode": "cnn_fixed", 
    # Training
    "num_workers": 4,
    "epoch": 50,
    "batchsize": batch_size,
    "scheduler_type": "CosineAnnealingWarmRestarts",
    "scheduler_kwargs": {"T_0": 30000},  # 15000 * 15 epoch // (batchsize=8)
    "scheduler_trigger": [1, "iteration"],
    "aug_kwargs": {
        "HorizontalFlip": {"p": 0.5},
        "ShiftScaleRotate": {"scale_limit": 0.15, "rotate_limit": 10, "p": 0.5},
        "RandomBrightnessContrast": {"p": 0.5},
        "CoarseDropout": {"max_holes": 8, "max_height": 25, "max_width": 25, "p": 0.5},
        "Blur": {"blur_limit": [3, 7], "p": 0.5},
        "Downscale": {"scale_min": 0.25, "scale_max": 0.9, "p": 0.3},
        "RandomGamma": {"gamma_limit": [80, 120], "p": 0.6},
    }
}

In [ ]:
import dataclasses
from pathlib import Path
import os
import pandas as pd
import numpy as np

# args = parse()
print("torch", torch.__version__)
flags = Flags().update(flags_dict)
print("flags", flags)
debug = flags.debug
outdir = Path(flags.outdir)
os.makedirs(str(outdir), exist_ok=True)
flags_dict = dataclasses.asdict(flags)
save_yaml(str(outdir / "flags.yaml"), flags_dict)

# # --- Read data ---
# inputdir = Path("/kaggle/input")
# datadir = inputdir / "vinbigdata-chest-xray-abnormalities-detection"
# imgdir = inputdir / flags.imgdir_name

# # Read in the data CSV files
# train = pd.read_csv(datadir / "train.csv")
# # sample_submission = pd.read_csv(datadir / 'sample_submission.csv')

torch 1.7.1+cu101
flags Flags(debug=False, outdir='results/tmp_debug', device='cuda:0', imgdir_name='vinbigdata-chest-xray-resized-png-256x256', seed=111, target_fold=0, label_smoothing=0.0, model_name='resnet50', model_mode='cnn_fixed', epoch=50, batchsize=32, valid_batchsize=16, num_workers=4, snapshot_freq=5, ema_decay=0.999, scheduler_type='CosineAnnealingWarmRestarts', scheduler_kwargs={'T_0': 30000}, scheduler_trigger=[1, 'iteration'], aug_kwargs={'HorizontalFlip': {'p': 0.5}, 'ShiftScaleRotate': {'scale_limit': 0.15, 'rotate_limit': 10, 'p': 0.5}, 'RandomBrightnessContrast': {'p': 0.5}, 'CoarseDropout': {'max_holes': 8, 'max_height': 25, 'max_width': 25, 'p': 0.5}, 'Blur': {'blur_limit': [3, 7], 'p': 0.5}, 'Downscale': {'scale_min': 0.25, 'scale_max': 0.9, 'p': 0.3}, 'RandomGamma': {'gamma_limit': [80, 120], 'p': 0.6}}, mixup_prob=-1.0)


#using timm

In [ ]:
import timm

from torch import nn
from torch.nn import Linear


class CNNFixedPredictor(nn.Module):
    def __init__(self, cnn: nn.Module, num_classes: int =2):
        super(CNNFixedPredictor, self).__init__()
        self.cnn = cnn
        self.lin1 = Linear(cnn.num_classes,num_classes)
        # self.tan1=nn.Tanh()
        # self.lin2 = Linear(int(cnn.num_classes/2),int(cnn.num_classes/3 ))
        self.dropout=nn.Dropout(p=0.25)
        # self.tan2=nn.Tanh()
        # self.lin3 = Linear(5,num_classes )
        # self.dropout=nn.Dropout(p=0.25)
        # self.tan2=nn.Tanh()
        # self.softmax=nn.Softmax(dim=1)
        # print("cnn.num_features", cnn.num_features)

        # We do not learn CNN parameters.
        # https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
        for param in self.cnn.parameters():
            param.requires_grad = False

    def forward(self, x):
        feat = self.cnn(x)
        feat=self.dropout(feat)
        feat=self.lin1(feat)
        # feat=self.tan1(feat)
        # feat=self.lin2(feat)
        
        # feat=self.tan2(feat)
        # feat=self.lin3(feat)
        # feat=self.softmax(feat)
        return feat

def build_predictor(model_name: str, model_mode: str = "normal"):
    if model_mode == "normal":
        # normal configuration. train all parameters.
        return timm.create_model(model_name, pretrained=True, num_classes=2, in_chans=3)
    elif model_mode == "cnn_fixed":
        # normal configuration. train all parameters.
        # https://rwightman.github.io/pytorch-image-models/feature_extraction/
        timm_model = timm.create_model(model_name, pretrained=True, num_classes=1000, in_chans=3)
        return CNNFixedPredictor(timm_model, num_classes=2)
    else:
        raise ValueError(f"[ERROR] Unexpected value model_mode={model_mode}")

import torch


def accuracy(y: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
    """Computes multi-class classification accuracy"""
    assert y.shape[:-1] == t.shape, f"y {y.shape}, t {t.shape} is inconsistent."
    pred_label = torch.max(y.detach(), dim=-1)[1]
    count = t.nelement()
    correct = (pred_label == t).sum().float()
    acc = correct / count
    return acc


def accuracy_with_logits(y: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
    """Computes multi-class classification accuracy"""
    assert y.shape == t.shape
    gt_label = torch.max(t.detach(), dim=-1)[1]
    return accuracy(y, gt_label)

import torch
import torch.nn.functional as F


def cross_entropy_with_logits(input, target, dim=-1):
    loss = torch.sum(- target * F.log_softmax(input, dim), dim)
    return loss.mean()

class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
      super().__init__()
      self.mse = nn.MSELoss()
      self.eps = eps
        
    def forward(self,yhat,y):
      # print("yhat",yhat)
      # 
      batch_size=yhat.shape[0]
      # print("batch_size",batch_size)
      loss = (self.mse(yhat,y) + self.eps)/batch_size

      return loss


In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
import pytorch_pfn_extras as ppe


class Classifier(nn.Module):
    """two class classfication"""

    def __init__(self, predictor, lossfun=torch.nn.L1Loss()):
        super().__init__()
        self.predictor = predictor
        self.lossfun = lossfun
        self.prefix = ""

    def forward(self, image,dffclt,dscrmn):
        outputs = self.predictor(image)
        outputs_dffclt=outputs[:,0]
        outputs_dscrmn=outputs[:,1]
        
        # print(outputs[:,0].shape)
        # print("dffclt",dffclt)
        # print("dscrmn",dscrmn)
        # targets=torch.nn.functional.one_hot(targets)
        loss_dffclt = self.lossfun(outputs_dffclt, dffclt)
        loss_dscrmn = self.lossfun(outputs_dscrmn, dscrmn)
        # print("loss_dffclt",loss_dffclt)
        # print("loss_dscrmn",loss_dscrmn)
        loss=loss_dscrmn+loss_dffclt
        metrics = {
            f"{self.prefix}loss_dffclt": loss_dffclt.item(),
            f"{self.prefix}loss_dscrmn": loss_dscrmn.item(),
            f"{self.prefix}loss": loss.item(),
            f"{self.prefix}rsme": torch.sqrt(loss)
        }
        # print(metrics)
        ppe.reporting.report(metrics, self)
        return loss, metrics

    def predict(self, data_loader):
        pred = self.predict_proba(data_loader)
        label = torch.argmax(pred, dim=1)
        return label

    def predict_proba(self, data_loader):
        device: torch.device = next(self.parameters()).device
        y_list = []
        self.eval()
        with torch.no_grad():
            for batch in data_loader:
                if isinstance(batch, (tuple, list)):
                    # Assumes first argument is "image"
                    batch = batch[0].to(device)
                else:
                    batch = batch.to(device)
                y = self.predictor(batch)
                y = torch.softmax(y, dim=-1)
                y_list.append(y)
        pred = torch.cat(y_list)
        return pred

In [ ]:
supported_models = timm.list_models()
print(f"{len(supported_models)} models are supported in timm.")
print(supported_models)

347 models are supported in timm.
['adv_inception_v3', 'cspdarknet53', 'cspdarknet53_iabn', 'cspresnet50', 'cspresnet50d', 'cspresnet50w', 'cspresnext50', 'cspresnext50_iabn', 'darknet53', 'densenet121', 'densenet121d', 'densenet161', 'densenet169', 'densenet201', 'densenet264', 'densenet264d_iabn', 'densenetblur121d', 'dla34', 'dla46_c', 'dla46x_c', 'dla60', 'dla60_res2net', 'dla60_res2next', 'dla60x', 'dla60x_c', 'dla102', 'dla102x', 'dla102x2', 'dla169', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn107', 'dpn131', 'eca_vovnet39b', 'ecaresnet18', 'ecaresnet50', 'ecaresnet50d', 'ecaresnet50d_pruned', 'ecaresnet101d', 'ecaresnet101d_pruned', 'ecaresnetlight', 'ecaresnext26tn_32x4d', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b1_pruned', 'efficientnet_b2', 'efficientnet_b2_pruned', 'efficientnet_b2a', 'efficientnet_b3', 'efficientnet_b3_pruned', 'efficientnet_b3a', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'efficientnet_b8', 'efficientnet_cc_b0_4e'

In [ ]:
from adversarial_project.pytorch.utils.ema import EMA
from adversarial_project.pytorch.utils.lrscheduler import LRScheduler

In [ ]:
from ignite.engine import Engine


def create_trainer(model, optimizer, device) -> Engine:
    model.to(device)

    def update_fn(engine, batch):
      # print(batch)
      image=batch["image"].float()
      dffclt=batch["Dffclt"].float()
      dscrmn=batch["Dscrmn"].float()
      img_path=batch["img_path"]
    
      batch=(image,dffclt,dscrmn)
      model.train()
      optimizer.zero_grad()
      loss, metrics = model(*[elem.to(device) for elem in batch])
      loss.backward()
      optimizer.step()
      return metrics
    trainer = Engine(update_fn)
    return trainer

In [ ]:
from torch import nn, optim
import pytorch_pfn_extras.training.extensions as E
from pytorch_pfn_extras.training import IgniteExtensionsManager
import logging
use_cuda=torch.cuda.is_available()
if use_cuda:
  device=torch.device("cuda")
else:
  device=torch.device("cpu")

predictor = build_predictor(model_name=flags.model_name, model_mode=flags.model_mode)
classifier = Classifier(predictor)
model = classifier

optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad], lr=5e-4)

# Train setup
trainer = create_trainer(model, optimizer, device)

ema = EMA(predictor, decay=flags.ema_decay)

def eval_func(**batch):
  # print(batch)
  image=batch["image"]
  dffclt=batch["Dffclt"].float()
  dscrmn=batch["Dscrmn"].float()
  
  batch=(image,dffclt,dscrmn)
  loss, metrics = model(*[elem.to(device) for elem in batch])
  # HACKING: report ema value with prefix.
  if flags.ema_decay > 0:
      classifier.prefix = "ema_"
      ema.assign()
      loss, metrics = model(*[elem.to(device) for elem in batch])
      ema.resume()
      classifier.prefix = ""

valid_evaluator = E.Evaluator(
    test_loader, model, progress_bar=True, eval_func=eval_func, device=device
)

# log_trigger = (10 if debug else 1000, "iteration")
log_trigger = (5, "epoch")
log_report = E.LogReport(trigger=log_trigger)
log_report=E.LogReport()
extensions = [
    log_report,
    E.ProgressBarNotebook(update_interval=10 if debug else 100),  # Show progress bar during training
    E.PrintReportNotebook(),  # Show "log" on jupyter notebook  
    E.ProgressBar(update_interval=10 if debug else 100),  # Show progress bar during training
#     # E.PrintReport(),  # Print "log" to terminal
#     E.FailOnNonNumber(),  # Stop training when nan is detected.
]
epoch = flags.epoch
models = {"main": model}
optimizers = {"main": optimizer}
manager = IgniteExtensionsManager(
    trainer, models, optimizers, epoch, extensions=extensions, out_dir=str(outdir),)
# Run evaluation for valid dataset in each epoch.
manager.extend(valid_evaluator)

# Save predictor.pt every epoch
manager.extend(
    E.snapshot_object(predictor, "predictor.pt"), trigger=(flags.snapshot_freq, "epoch")
)
# Check & Save best validation predictor.pt every epoch
# manager.extend(E.snapshot_object(predictor, "best_predictor.pt"),
#                trigger=MinValueTrigger("validation/module/nll",
#                trigger=(flags.snapshot_freq, "iteration")))

# --- lr scheduler ---
if flags.scheduler_type != "":
    scheduler_type = flags.scheduler_type
    print(f"using {scheduler_type} scheduler with kwargs {flags.scheduler_kwargs}")
    manager.extend(
        LRScheduler(optimizer, scheduler_type, flags.scheduler_kwargs),
        trigger=flags.scheduler_trigger,
    )

manager.extend(E.observe_lr(optimizer=optimizer), trigger=log_trigger)

if flags.ema_decay > 0:
    # Exponential moving average
    manager.extend(lambda manager: ema(), trigger=(1, "iteration"))

    def save_ema_model(manager):
        ema.assign()
        torch.save(predictor.state_dict(), outdir / "predictor_ema.pt")
        ema.resume()

    manager.extend(save_ema_model, trigger=(flags.snapshot_freq, "epoch"))

_ = trainer.run(train_loader, max_epochs=flags.epoch)

using CosineAnnealingWarmRestarts scheduler with kwargs {'T_0': 30000}


HTML(value='')

validation [..................................................]  0.00%
         0 iterations
       inf iters/sec. Estimated time to finish: 0:00:00.
validation [##................................................]  4.35%
         1 iterations
    2.2019 iters/sec. Estimated time to finish: 0:00:09.991388.
validation [####..............................................]  8.70%
         2 iterations
    2.1748 iters/sec. Estimated time to finish: 0:00:09.656276.
validation [######............................................] 13.04%
         3 iterations
    2.1834 iters/sec. Estimated time to finish: 0:00:09.159978.
validation [########..........................................] 17.39%
         4 iterations
    2.1697 iters/sec. Estimated time to finish: 0:00:08.756955.
validation [##########........................................] 21.74%
         5 iterations
    1.9961 iters/sec. Estimated time to finish: 0:00:09.017606.
validation [#############.....................................] 2

In [ ]:
#@title quinto intento con resnet y normalización de los valores de salida
# if True:
torch.save(predictor.state_dict(), outdir / "predictor_last.pt")
df = log_report.to_dataframe()
df.to_csv(outdir / "log.csv", index=False)
df

,main/loss_dffclt,main/loss_dscrmn,main/loss,main/rsme,validation/main/loss_dffclt,validation/main/loss_dscrmn,validation/main/loss,validation/main/rsme,validation/main/ema_loss_dffclt,validation/main/ema_loss_dscrmn,validation/main/ema_loss,validation/main/ema_rsme,epoch,iteration,elapsed_time,lr
0,0.135629,0.468973,0.604602,0.776146,0.101208,0.359417,0.460625,0.678107,0.103149,0.374366,0.477515,0.690419,1,39,25.265190,NaN
1,0.127893,0.340563,0.468456,0.683651,0.095977,0.286796,0.382773,0.617580,0.097835,0.300640,0.398475,0.630144,2,78,50.644748,NaN
2,0.124175,0.295080,0.419255,0.647052,0.081927,0.246751,0.328678,0.572266,0.084577,0.257704,0.342281,0.584108,3,117,75.988359,NaN
3,0.117609,0.264285,0.381894,0.616997,0.079806,0.228799,0.308604,0.555025,0.080455,0.234471,0.314926,0.560622,4,156,101.391339,NaN
4,0.110272,0.245547,0.355818,0.595532,0.075322,0.205818,0.281140,0.529554,0.077598,0.214602,0.292200,0.539917,5,195,127.055805,0.000100
5,0.104296,0.231679,0.335975,0.578988,0.071019,0.198597,0.269616,0.518360,0.073977,0.204332,0.278309,0.526634,6,234,153.399363,NaN
6,0.099388,0.229373,0.328761,0.572345,0.067701,0.186122,0.253822,0.502781,0.070123,0.191326,0.261449,0.510358,7,273,179.011901,NaN
7,0.100695,0.211739,0.312434,0.558202,0.065668,0.180597,0.246266,0.495546,0.065839,0.188498,0.254337,0.503735,8,312,204.704053,NaN
8,0.092902,0.213336,0.306237,0.552636,0.062249,0.176140,0.238389,0.487340,0.063654,0.179871,0.243525,0.492568,9,351,230.272210,NaN
9,0.087516,0.206583,0.294099,0.541695,0.058565,0.174189,0.232754,0.481770,0.061316,0.176861,0.238176,0.487364,10,390,255.947095,0.000100


In [ ]:
#@title cuarto intento con densenet sin fix
if False:
  torch.save(predictor.state_dict(), outdir / "predictor_last.pt")
  df = log_report.to_dataframe()
  df.to_csv(outdir / "log.csv", index=False)
  df

In [ ]:
#@title tercer intento con densenet con capa de activacion intermedia tangente  
if False:
  torch.save(predictor.state_dict(), outdir / "predictor_last.pt")
  df = log_report.to_dataframe()
  df.to_csv(outdir / "log.csv", index=False)
  df

,main/loss_dffclt,main/loss_dscrmn,main/loss,validation/main/loss_dffclt,validation/main/loss_dscrmn,validation/main/loss,validation/main/ema_loss_dffclt,validation/main/ema_loss_dscrmn,validation/main/ema_loss,epoch,iteration,elapsed_time,lr
0,1.935228,0.467696,2.402923,1.455174,0.460090,1.915265,1.457090,0.463271,1.920361,1,39,41.610673,NaN
1,1.999150,0.425712,2.424862,1.602514,0.471291,2.073805,1.603154,0.472405,2.075558,2,78,83.602510,NaN
2,2.005693,0.426466,2.432160,1.435448,0.422324,1.857772,1.435658,0.423573,1.859231,3,117,125.402080,NaN
3,1.996615,0.496123,2.492738,1.473769,0.437156,1.910925,1.473916,0.434981,1.908897,4,156,167.237875,NaN
4,1.957753,0.425233,2.382986,1.183028,0.408212,1.591240,1.182699,0.403723,1.586422,5,195,208.914842,0.001000
5,1.998767,0.463866,2.462634,1.450693,0.458861,1.909553,1.449947,0.454139,1.904086,6,234,251.297923,NaN
6,1.907610,0.463096,2.370707,1.850285,0.412462,2.262746,1.848421,0.408418,2.256839,7,273,293.032560,NaN
7,1.967215,0.448547,2.415762,1.449115,0.447213,1.896329,1.445215,0.454833,1.900048,8,312,334.764905,NaN
8,1.605087,0.475085,2.080172,1.460700,0.395368,1.856067,1.458883,0.391797,1.850680,9,351,376.609354,NaN
9,1.787077,0.402807,2.189885,1.450659,0.419326,1.869986,1.446157,0.424368,1.870525,10,390,418.512728,0.001000


In [ ]:
#@title segundo intento con resnet18 con capa de activacion intermedia tangente  
if False:
  torch.save(predictor.state_dict(), outdir / "predictor_last.pt")
  df = log_report.to_dataframe()
  df.to_csv(outdir / "log.csv", index=False)
  df

,main/loss_dffclt,main/loss_dscrmn,main/loss,validation/main/loss_dffclt,validation/main/loss_dscrmn,validation/main/loss,validation/main/ema_loss_dffclt,validation/main/ema_loss_dscrmn,validation/main/ema_loss,epoch,iteration,elapsed_time,lr
0,1.911410,0.442891,2.354301,1.451263,0.396216,1.847479,1.450232,0.398184,1.848416,1,39,27.333997,NaN
1,1.725100,0.433048,2.158148,1.247275,0.444096,1.691372,1.247119,0.445462,1.692581,2,78,54.722007,NaN
2,2.060130,0.463454,2.523585,1.417985,0.407187,1.825172,1.417829,0.401818,1.819647,3,117,82.181611,NaN
3,1.684726,0.425316,2.110042,1.449116,0.414951,1.864066,1.450405,0.412856,1.863261,4,156,110.027626,NaN
4,2.000509,0.421669,2.422178,1.450823,0.475482,1.926306,1.450835,0.476914,1.927749,5,195,137.628296,0.001000
5,1.973859,0.416062,2.389921,1.429845,0.413886,1.843731,1.430108,0.411346,1.841454,6,234,165.605877,NaN
6,1.992430,0.427866,2.420296,5.760270,0.476209,6.236479,5.757783,0.477435,6.235217,7,273,193.315965,NaN
7,1.888994,0.439833,2.328827,1.453804,0.465179,1.918983,1.452057,0.462463,1.914520,8,312,220.843442,NaN
8,1.966965,0.410715,2.377680,1.476410,0.452813,1.929223,1.473458,0.450669,1.924127,9,351,248.483603,NaN
9,1.988414,0.454864,2.443278,1.470847,0.415523,1.886370,1.470379,0.411335,1.881714,10,390,276.256386,0.001000


In [ ]:
#@title primer intento con resnet18 sin capa de activacion intermedia tangente
if False:
  torch.save(predictor.state_dict(), outdir / "predictor_last.pt")
  df = log_report.to_dataframe()
  df.to_csv(outdir / "log.csv", index=False)
  df

,main/loss_dffclt,main/loss_dscrmn,main/loss,validation/main/loss_dffclt,validation/main/loss_dscrmn,validation/main/loss,validation/main/ema_loss_dffclt,validation/main/ema_loss_dscrmn,validation/main/ema_loss,epoch,iteration,elapsed_time,lr
0,1.940317,0.458919,2.399236,1.476007,0.464717,1.940724,1.474364,0.469498,1.943862,1,39,27.435280,NaN
1,2.037688,0.521563,2.559251,1.476930,0.468484,1.945414,1.475184,0.465672,1.940856,2,78,54.982415,NaN
2,1.995424,0.438112,2.433536,1.441509,0.474366,1.915875,1.443134,0.474419,1.917553,3,117,82.522694,NaN
3,2.001976,0.427386,2.429362,1.455987,0.400705,1.856691,1.456345,0.396684,1.853030,4,156,109.852696,NaN
4,1.876562,0.425738,2.302300,1.479081,0.447178,1.926259,1.477641,0.445570,1.923211,5,195,137.201215,0.001000
5,1.721427,0.419565,2.140992,1.473486,0.415550,1.889036,1.472347,0.406820,1.879168,6,234,164.992015,NaN
6,2.009517,0.425369,2.434886,1.429965,0.416671,1.846636,1.430239,0.415859,1.846098,7,273,192.502991,NaN
7,1.997993,0.480975,2.478968,1.441184,0.459533,1.900718,1.440397,0.453081,1.893477,8,312,219.995272,NaN
8,1.987344,0.465946,2.453291,1.486898,0.470157,1.957055,1.482586,0.463271,1.945857,9,351,247.430904,NaN
9,1.944492,0.429083,2.373575,1.472857,0.420793,1.893651,1.470825,0.425427,1.896252,10,390,274.918924,0.001000


Mostrando datos reales

In [ ]:
device

device(type='cuda')

In [ ]:
# --- Prediction ---

print("Training done! Start prediction...")
# valid data
all_valid_pred_df=pd.DataFrame()
for batch in test_loader:

  images=batch["image"].float()
  dffclt=batch["Dffclt"].float()
  dscrmn=batch["Dscrmn"].float()
  img_paths=batch["img_path"]

  with torch.no_grad():
    valid_pred=classifier.predictor(images.to(device)).cpu().numpy()

    # valid_pred=torch.nn.Softmax(1)(valid_pred).cpu().numpy()
  valid_pred=np.round(valid_pred, 5)

  # valid_pred=softmax(valid_pred)
  valid_pred_df = pd.DataFrame({
      "image_id": img_paths,
      "norm_Dffclt":dffclt,
      "norm_Dscrmn":dscrmn,
      "PRED_Dffclt": valid_pred[:, 0],
      "PRED_Dscrmn": valid_pred[:, 1],
  })

  all_valid_pred_df=pd.concat([all_valid_pred_df,valid_pred_df])


all_valid_pred_df.to_csv(outdir/"valid_pred.csv", index=False)



Training done! Start prediction...


In [ ]:
all_valid_pred_df

,image_id,norm_Dffclt,norm_Dscrmn,PRED_Dffclt,PRED_Dscrmn
0,/content/imagenet_validation/ILSVRC2012_val_00...,0.165067,0.442428,0.10825,0.13775
1,/content/imagenet_validation/ILSVRC2012_val_00...,0.164684,0.428548,0.20268,0.10464
2,/content/imagenet_validation/ILSVRC2012_val_00...,0.165296,0.432729,0.24846,0.63904
3,/content/imagenet_validation/ILSVRC2012_val_00...,0.165035,0.456066,0.11887,0.27744
4,/content/imagenet_validation/ILSVRC2012_val_00...,0.165233,0.429886,0.20713,0.47007
...,...,...,...,...,...
24,/content/imagenet_validation/ILSVRC2012_val_00...,0.165208,0.420164,0.18768,0.13323
25,/content/imagenet_validation/ILSVRC2012_val_00...,0.165741,0.426356,0.13797,0.37099
26,/content/imagenet_validation/ILSVRC2012_val_00...,0.165863,0.758046,0.21834,0.63704
27,/content/imagenet_validation/ILSVRC2012_val_00...,0.165110,0.462466,0.08865,0.17743


In [ ]:
dffclt

tensor([ 1.7955e-01, -1.2686e+00,  7.6312e+00,  8.6879e-01, -1.6772e+00,
        -2.1564e+00, -2.0828e+00, -1.1764e+00, -4.3628e-01,  1.5604e+00,
         2.3040e+00, -2.8933e-02, -1.4847e+00, -1.0124e+00, -4.3524e+00,
        -1.3052e+00, -4.1657e+00,  1.7087e+00, -3.2220e+00,  1.2204e+00,
        -2.7649e+00, -4.5684e+00,  9.8494e-01, -3.9101e+00,  6.2800e-01,
         7.2123e+00, -2.4332e+00, -7.6200e-01, -1.3405e+00, -1.0943e+01,
         1.2714e+02, -7.7274e-01])

In [ ]:
loss = nn.MarginRankingLoss()
input1 = torch.tensor([1,0,0])
input2 = torch.tensor([1,1,1])
target = torch.tensor([1,1,1])
output = loss(input1, input2, target)
print(input1)
print(input2)
print(target)
print(output)
# output.backward()

tensor([1, 0, 0])
tensor([1, 1, 1])
tensor([1, 1, 1])
tensor(0.6667)


tensor([0, 0, 0])
tensor([-1,  1,  1])
tensor([1, 1, 1])
tensor(0.6667)
